In [ ]:
# находим корень проекта a4s-eval 
from pathlib import Path

def get_project_root() -> Path:
    """
    Возвращает корень проекта a4s-eval вне зависимости от того, 
    где запускается код: из тестов, из a4s_eval, из ноутбуков.
    """
    current = Path.cwd().resolve()

    # Ищем директорию, где находится папка a4s_eval
    for parent in [current] + list(current.parents):
        if (parent / "a4s_eval").exists():
            return parent

    # Fallback — если запускается из .py, где есть __file__
    try:
        file_path = Path(__file__).resolve()
        for parent in [file_path] + list(file_path.parents):
            if (parent / "a4s_eval").exists():
                return parent
    except NameError:
        pass  # __file__ не существует (например, в Jupyter)

    raise RuntimeError("Не удалось найти корень проекта a4s-eval")


In [4]:

import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tests.additional_files.train_regression_model import run
from a4s_eval.metric_registries.prediction_metric_registry import prediction_metric_registry

# Получаем реальные данные
y_test, y_pred = run()

# Наши функции из registry
mae_func = prediction_metric_registry.get_functions()["mae"]
mse_func = prediction_metric_registry.get_functions()["mse"]
r2_func = prediction_metric_registry.get_functions()["r2"]

# Dummy dataset, как в тесте
Dataset = type("Dataset", (), {"y": y_test})
dataset = Dataset()

# Наши вычисления
mae_ours = mae_func(None, None, dataset, y_pred)[0]
mse_ours = mse_func(None, None, dataset, y_pred)[0]
r2_ours = r2_func(None, None, dataset, y_pred)[0]

# sklearn значения
mae_sk = mean_absolute_error(y_test, y_pred)
mse_sk = mean_squared_error(y_test, y_pred)
r2_sk = r2_score(y_test, y_pred)

print("=== OUR METRICS ===")
print("MAE:", mae_ours)
print("MSE:", mse_ours)
print("R2:",  r2_ours)

print("\n=== SKLEARN METRICS ===")
print("MAE:", mae_sk)
print("MSE:", mse_sk)
print("R2:",  r2_sk)

print("\n=== ABS DIFFERENCES ===")
print("MAE diff:", abs(mae_ours - mae_sk))
print("MSE diff:", abs(mse_ours - mse_sk))
print("R2 diff:",  abs(r2_ours - r2_sk))


=== OUR METRICS ===
MAE: 0.024490033253484568
MSE: 0.0015794964938804028
R2: 0.9980326174775115

=== SKLEARN METRICS ===
MAE: 0.024490033253484568
MSE: 0.0015794964938804028
R2: 0.9980326174775115

=== ABS DIFFERENCES ===
MAE diff: 0.0
MSE diff: 0.0
R2 diff: 0.0


In [ ]:
""""1️⃣ Визуальный контроль
Нарисуй график y_true и y_pred вместе.
На графике отметь пики из DeDiPeak (peaks_true и peaks_pred).
Проверь, совпадают ли пики, и насколько сильно они смещены во времени и по амплитуде.
Если пики почти совпадают — метрика должна быть маленькой, если сильно расходятся — большая."""

import matplotlib.pyplot as plt

y_true, y_pred = run(n_rows=1000)
value, peaks = P3E(model=None, X=None, dataset=None, y_pred=y_pred, y_true=y_true)

plt.figure(figsize=(12,4))
plt.plot(y_true, label='y_true')
plt.plot(y_pred, label='y_pred')
plt.scatter(peaks['peaks_true'], y_true[peaks['peaks_true']], color='green', label='True peaks')
plt.scatter(peaks['peaks_pred'], y_pred[peaks['peaks_pred']], color='red', label='Predicted peaks')
plt.legend()
plt.show()


NameError: name 'run' is not defined

In [ ]:
"""2️⃣ Мини-примеры
Создай очень маленький массив, где пики известны руками, и запусти метрику на нём. 
Здесь ты сам видишь, где пики и какое расстояние получилось.
Если результат соответствует твоему ожиданию, значит функция работает корректно."""

import numpy as np
y_true = np.array([0, 1, 0, 0, 1, 0])
y_pred = np.array([0, 0.9, 0, 0, 1.1, 0])

value, peaks = P3E(model=None, X=None, dataset=None, y_pred=y_pred, y_true=y_true)
print(peaks)
print("DeDiPeak:", value)


In [ ]:
"""3️⃣ Проверка логики метрики
Посмотри на код dedipeak_metric.py.
Убедись, что P3E считает именно расстояние между найденными пиками (peaks_true ↔ peaks_pred) с нужной нормализацией.
Можно добавить print внутри функции для отладки: какие пики выбраны и какие расстояния считаются."""

In [ ]:
#код для проверки загрузки и очистки данных, чтобы сравнить насколько датасет уменьшился после очистки 
import pandas as pd

path = "tests/data/household_power_consumption.txt"

# Загружаем сырые данные
df_raw = pd.read_csv(
    path,
    sep=";",
    low_memory=False,
    na_values=["?"]
)

raw_count = len(df_raw)

# Обрабатываем — создаём Datetime
df_raw["Datetime"] = pd.to_datetime(
    df_raw["Date"] + " " + df_raw["Time"],
    dayfirst=True,
    errors="coerce"
)

# Удаляем NaN/NaT
df_clean = df_raw.dropna()
clean_count = len(df_clean)

print(f"Total rows before cleaning: {raw_count}")
print(f"Rows after cleaning: {clean_count}")
print(f"Rows removed: {raw_count - clean_count}")
print(f"Percentage removed: {100 * (raw_count - clean_count) / raw_count:.2f}%")
